# Photometric Redshifts of Galaxies

## 1(a) Feature Extraction

To understand Feature Extraction, it is imperative that we understand what 'feature' means. As I understand it, feature is analogues to 'variable' used in Physics. Thus, a feature is a set of input variables that can be measured independently. In fact, measurement of variables is essential to predict some outcome based on relationships between them. However, there can also be an extention to this definition. New features can be constructed from a combination of other pre-existing features using various mathematical tools.

In this sense, we can now begin to define feature extraction. In this context, we have been given a set of data where one column namely the redshift is thought of to be a combination of four other columns which are colours that can be measured. Hence, we term the four colour combinations as features. But, it might turn out that redshift might not depend on combination of all these features i.e some features might be less important or might be dependant on other features or redshift might not be directly related to these features, but in turn is related to an operation done on those features i.e newly constructed features. This entire process of figuring out the 'true variables' is called as Feature Extraction.

Note: This is fundamentally different from Feature Selection because Feature Selection assumes that we already know a list of features and hence need to know the best of them, while Feature Extraction doesn't assume anything.

## 1(b+c) Photometric Estimator based on Linear Regression

First the data in the votables are transferred to csv files which are easier to handle (see q1 also for more reasons). For this, the python file named vottocsv.py is used. Then we import the necessary modules.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn import neighbors
from sklearn.ensemble import RandomForestRegressor

sns.set()
%matplotlib inline

Now we fetch the data.

In [2]:
train = pd.read_csv('Traindata.csv')
test = pd.read_csv('Testdata.csv')

Now select the columns for training the model. We also extract the test data as Ygen and Xgen.

In [5]:
X = train[['u-g','g-r','r-i','i-z']]
Y = train[['z_spec']]
Ygen = test[['z_spec']]
Xgen = test[['u-g','g-r','r-i','i-z']]

Getting a linear algorithm using the entire train dataset will cause an overfit of the data by the algorithm to a very strong extent. Hence, instead we need to train this dataset by using chunks of the first dataset to first train the algorithm and then try to fit it with another chunk of the same dataset to reduce the overall overfitting. However, the model still remains biased to this dataset.

For tuning the model, we now divide the train data into sets of 'train' and 'test' datasets using k-fold cross validation. The code is inspired from the code used in the assignments. Then we use a Linear Regression model to fit the train data sets and predict values for the test data sets. This then gives us the 'Training Error' for the first data set. However, this is not the true error. Since, we use exclusively one dataset to train our algorithm, the algorithmn might be biased towards fitting the first set of data only. Hence, a second set of data taken at a different time or place can say us how well this model actually works. This is done by applying the regressor to Xgen from the second dataset and comparing it with Ygen. This error is called 'Generalization Error'. The training error is shown first followed by the generalization error which is obatained by testing the trained model on the second dataset.

In [17]:
k = KFold(n_splits = 5)

for train_index, test_index in k.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index] #.iloc is added to take care of indices to dataframe incompatibility
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
    l = LinearRegression()
    l.fit(X_train, Y_train) 
    print(l.intercept_, l.coef_, l.score(X_train,Y_train)) # Parameters of the fit
    t = (Y_test-l.predict(X_test))/(1+Y_test)
    t1 = t.abs()
    Error = t1.median()
    print  'Training Error is', Error['z_spec'] # The training error
    g = (Ygen-l.predict(Xgen))/(1+Ygen)
    g1 = g.abs()
    Errorg = g1.median()
    print  'Generalization Error is', Errorg['z_spec'] # The generalization error

('TRAIN:', array([14862, 14863, 14864, ..., 74306, 74307, 74308]), 'TEST:', array([    0,     1,     2, ..., 14859, 14860, 14861]))
(array([-0.22289139]), array([[-0.02558786,  0.11659748,  0.67506641,  0.00336796]]), 0.77273877318220374)
Training Error is 0.0152035390976
Generalization Error is 0.015156971496
('TRAIN:', array([    0,     1,     2, ..., 74306, 74307, 74308]), 'TEST:', array([14862, 14863, 14864, ..., 29721, 29722, 29723]))
(array([-0.22042711]), array([[-0.02642282,  0.11579633,  0.67301541,  0.00735718]]), 0.76690562554573694)
Training Error is 0.0144620329545
Generalization Error is 0.0151699789005
('TRAIN:', array([    0,     1,     2, ..., 74306, 74307, 74308]), 'TEST:', array([29724, 29725, 29726, ..., 44583, 44584, 44585]))
(array([-0.22220995]), array([[-0.02592871,  0.11580501,  0.67458606,  0.0073301 ]]), 0.76990746597627346)
Training Error is 0.0147754956634
Generalization Error is 0.0151676828116
('TRAIN:', array([    0,     1,     2, ..., 74306, 74307, 7430

It is easily seen that the train and test error are comparable and the parameters for the model are not large. Thus, we don't need a regularization technique like ridge or lasso. It also means the training has been done well and the data actually follows a linear trend.

## 1(d) Non-linear Photometric estimator

Now that we have tried out a Linear Estimator, let's use a non-linear estimator of our choice. I use k-Nearest Neighbours regressor. I personally favour this because I understand it more clearly among all non-linear methods. However, there is also another reason to use this. The features we consider have only 4 dimensions which isn't much and all features can very well be assumed to be continuous. Knn is very effective in these situations.

For knn I take a high number of neighbours for a good averaging and I weigh all of them by distance, so that the closer points get more prefrence. This is partly inspired by the fact that I already know that the data trend is fairly linear but still has a significant amount of variation from a linear trend. This is known from seeing the linear scores (l.score(X,Y) computed in the linear model iterations with almost all of them being close to a value of 0.8.

In [23]:
k = KFold(n_splits = 5)

for train_index, test_index in k.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index] #.iloc is added to take care of indices to dataframe incompatibility
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
    knn = neighbors.KNeighborsRegressor(n_neighbors = 10, weights = 'distance') 
    knn.fit(X_train, Y_train)
    t = (Y_test-knn.predict(X_test))/(1.0+Y_test)
    t1 = t.abs()
    Error = t1.median()
    print 'Training Error is', Error['z_spec'] # Training Error
    gk = (Ygen-knn.predict(Xgen))/(1+Ygen)
    gk1 = gk.abs()
    Errorgk = gk1.median()
    print  'Generalization Error is', Errorgk['z_spec'] # Generalization Error

('TRAIN:', array([14862, 14863, 14864, ..., 74306, 74307, 74308]), 'TEST:', array([    0,     1,     2, ..., 14859, 14860, 14861]))
Training Error is 0.0127386609135
Generalization Error is 0.0127799120494
('TRAIN:', array([    0,     1,     2, ..., 74306, 74307, 74308]), 'TEST:', array([14862, 14863, 14864, ..., 29721, 29722, 29723]))
Training Error is 0.0120293279604
Generalization Error is 0.0127819611231
('TRAIN:', array([    0,     1,     2, ..., 74306, 74307, 74308]), 'TEST:', array([29724, 29725, 29726, ..., 44583, 44584, 44585]))
Training Error is 0.0124790116275
Generalization Error is 0.0127756396825
('TRAIN:', array([    0,     1,     2, ..., 74306, 74307, 74308]), 'TEST:', array([44586, 44587, 44588, ..., 59445, 59446, 59447]))
Training Error is 0.0122587697669
Generalization Error is 0.01283689562
('TRAIN:', array([    0,     1,     2, ..., 59445, 59446, 59447]), 'TEST:', array([59448, 59449, 59450, ..., 74306, 74307, 74308]))
Training Error is 0.0138375971945
Generalizati

It is seen that the training errors are considerably lesser than values with a linear model. Not only that, the generalization errors show that the non-linear model also holds up very well. The generalization errors obtained here are less than the generalization errors obtained for the linear case.

## 1(e) Advantages and Disadvantages of KNN over Linear Estimator

### Advantages

1. KNN estimator is very flexible as it is non-parametric. So, almost all values in the training set are considered for fitting. This can also be a disadvantage as we will see later.
2. There are only 4 features and the data is fairly linear, so careful use of the non-parametric feature will better constrain the model while training. In fact, training by KNN is actually more effective here. If we would have fit the entire train file without weighting and with less neighbours, the training error will be very low (See Appendix). But, it will overfit the data to a very large amount, so the generalization error is larger.

### Disadvantages

1. The flexibilty of the approach also means that it is prone to interference from outlier data. This would mean that we would be in a sense overfitting the data if we don't take considerable care in selecting the number of neighbours and the weighting.
2. The KNN estimator is also computationally more intensive. It searches for neighbours and gives weights by calculating the distance. This takes longer the more features there are. Fortunately, we have only 4 features.

## Appendix

KNN using the whole train set and then applied to the test dataset.

In [25]:
knn = neighbors.KNeighborsRegressor(n_neighbors = 10, weights = 'distance') 
knn.fit(X, Y)
ty = (Y-knn.predict(X))/(1.0+Y)
ty1 = ty.abs()
Errory = ty1.median()
print 'Training Error is', Errory['z_spec'] # Training Error
gyk = (Ygen-knn.predict(Xgen))/(1+Ygen)
gyk1 = gyk.abs()
Errorgyk = gyk1.median()
print  'Generalization Error is', Errorgyk['z_spec'] # Generalization Error

Training Error is 0.0
Generalization Error is 0.0127373528291


It is easy to see that the data fits the training data almost perfectly but it doesn't do the same for the test dataset. This is because it overfits the train data, which is exactly what we don't want.

## References

The material uses the scikit-learn software package version 0.19.1. I also took help from the online documentation of the packages used to write my code. In addition, code from the assignment was also liberally used. 